# Cifar10 Classification

This kernel was built for cifar10 classification.  

You can use this kernel to become familiar with Pytorch and CNN.  

You can easily change the dataset and CNNs you use to build another baseline.

In [1]:
# Preparations
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os, sys
import copy

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

Using CUDA


In [2]:
# Load the data. Calculate the images in each set and print the number of class
data_dir = 'cifar10'
Train = 'train'
Test = 'test'

# Transform image data to suitable size with data augmentation
data_transforms = {
    Train: transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),

    ]),
    Test: transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
}

# Specify the datasets
image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x), 
        transform=data_transforms[x]
    )
    for x in [Train, Test]
    }

# Build the dataloader
dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=64,
        shuffle=True, num_workers=4
    )
    for x in [Train, Test]
}

dataset_sizes = {x: len(image_datasets[x]) for x in [Train, Test]}

for x in [Train, Test]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))
    
print("Classes: ")
class_names = image_datasets[Train].classes 
print(image_datasets[Train].classes)

Loaded 50000 images under train
Loaded 10000 images under test
Classes: 
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [3]:
# Computes the accuracy over the k top predictions for the specified values of k
def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [4]:
# Computes and stores the average and current value
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [5]:
# Define the training process and return the accuracy
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (inputs, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        target = target.cuda()
        input_var = torch.autograd.Variable(inputs.cuda())
        target_var = torch.autograd.Variable(target.cuda())

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)
        losses.update(loss.data.item(), inputs.size(0))
        top1.update(prec1[0][0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            curr_lr = optimizer.param_groups[0]['lr']
            print('Epoch: [{0}/{1}][{2}/{3}]\t'
                  'LR: {4}\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                   epoch, 200, i, len(train_loader), curr_lr,
                   batch_time=batch_time, data_time=data_time, loss=losses, top1=top1))

    print(' * Training Prec@1 {top1.avg:.3f}'.format(top1=top1))
    
    return top1.avg

In [6]:
# Define the validation process and return the accuracy
def validate(test_loader, model, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (inputs, target) in enumerate(test_loader):
        target = target.cuda()
        input_var = torch.autograd.Variable(inputs.cuda(), volatile=True)
        target_var = torch.autograd.Variable(target.cuda(), volatile=True)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)
        losses.update(loss.data.item(), inputs.size(0))
        top1.update(prec1[0][0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                   i, len(test_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Validation Prec@1 {top1.avg:.3f}'.format(top1=top1))

    return top1.avg

In [7]:
# Specify the training model

Resnet18 = models.resnet18(pretrained = True)
Resnet18.fc = nn.Linear(512, len(class_names))
Resnet18.cuda()

# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(Resnet18.parameters(), lr = 0.01, momentum=0.9)

# Store the training and validation error
valError = [0.9]
trainError = [0.9]

In [8]:
# Train the model and evaluate the model after every epoch
for epoch in range(30):
    if epoch == 10:
        for param_group in optimizer.param_groups:
            param_group['lr'] *= 0.1

    # train for one epoch
    start = time.time()
    predTrain = train(dataloaders[Train], Resnet18, criterion, optimizer, epoch)
    trainError.append(1 - predTrain.item()/100)
    end = time.time()
    print('Training Time {:.3f}\t'.format(end-start))

    # evaluate on validation set
    start = time.time()
    predVal = validate(dataloaders[Test], Resnet18, criterion)
    valError.append(1 - predVal.item()/100)
    end = time.time()
    print('Validation Time {:.3f}\t'.format(end-start))

Epoch: [0/200][0/782]	LR: 0.01	Time 5.173 (5.173)	Data 3.943 (3.943)	Loss 2.5439 (2.5439)	Prec@1 10.938 (10.938)
Epoch: [0/200][100/782]	LR: 0.01	Time 0.358 (0.408)	Data 0.005 (0.046)	Loss 1.2828 (1.4058)	Prec@1 53.125 (50.619)
Epoch: [0/200][200/782]	LR: 0.01	Time 0.359 (0.385)	Data 0.005 (0.027)	Loss 0.9224 (1.2397)	Prec@1 65.625 (56.841)
Epoch: [0/200][300/782]	LR: 0.01	Time 0.364 (0.378)	Data 0.005 (0.020)	Loss 1.5147 (1.1451)	Prec@1 60.938 (60.263)
Epoch: [0/200][400/782]	LR: 0.01	Time 0.365 (0.375)	Data 0.005 (0.017)	Loss 0.8399 (1.0796)	Prec@1 68.750 (62.722)
Epoch: [0/200][500/782]	LR: 0.01	Time 0.362 (0.373)	Data 0.005 (0.015)	Loss 0.8157 (1.0278)	Prec@1 71.875 (64.533)
Epoch: [0/200][600/782]	LR: 0.01	Time 0.362 (0.371)	Data 0.005 (0.014)	Loss 1.0824 (0.9850)	Prec@1 65.625 (66.033)
Epoch: [0/200][700/782]	LR: 0.01	Time 0.370 (0.371)	Data 0.005 (0.013)	Loss 0.9652 (0.9559)	Prec@1 68.750 (67.009)
 * Training Prec@1 67.664
Training Time 289.359	


c:\users\vogelyu\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:13: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  del sys.path[0]
c:\users\vogelyu\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Test: [0/157]	Time 4.029 (4.029)	Loss 0.7469 (0.7469)	Prec@1 73.438 (73.438)
Test: [100/157]	Time 0.123 (0.163)	Loss 0.8088 (0.7509)	Prec@1 71.875 (74.072)
 * Validation Prec@1 73.890
Validation Time 23.377	
Epoch: [1/200][0/782]	LR: 0.01	Time 4.120 (4.120)	Data 3.968 (3.968)	Loss 0.5472 (0.5472)	Prec@1 84.375 (84.375)
Epoch: [1/200][100/782]	LR: 0.01	Time 0.368 (0.402)	Data 0.005 (0.046)	Loss 0.6875 (0.7110)	Prec@1 70.312 (75.557)
Epoch: [1/200][200/782]	LR: 0.01	Time 0.366 (0.384)	Data 0.006 (0.027)	Loss 0.7878 (0.7204)	Prec@1 73.438 (75.008)
Epoch: [1/200][300/782]	LR: 0.01	Time 0.363 (0.378)	Data 0.005 (0.020)	Loss 0.7187 (0.7140)	Prec@1 75.000 (75.114)
Epoch: [1/200][400/782]	LR: 0.01	Time 0.368 (0.375)	Data 0.005 (0.017)	Loss 0.5283 (0.7032)	Prec@1 82.812 (75.736)
Epoch: [1/200][500/782]	LR: 0.01	Time 0.368 (0.373)	Data 0.005 (0.015)	Loss 0.6493 (0.7003)	Prec@1 78.125 (75.805)
Epoch: [1/200][600/782]	LR: 0.01	Time 0.363 (0.372)	Data 0.005 (0.014)	Loss 0.9543 (0.6979)	Prec@1 64.06

 * Training Prec@1 84.088
Training Time 267.448	
Test: [0/157]	Time 4.020 (4.020)	Loss 0.4223 (0.4223)	Prec@1 84.375 (84.375)
Test: [100/157]	Time 0.132 (0.168)	Loss 0.7632 (0.5019)	Prec@1 71.875 (82.766)
 * Validation Prec@1 83.060
Validation Time 24.229	
Epoch: [8/200][0/782]	LR: 0.01	Time 4.106 (4.106)	Data 3.954 (3.954)	Loss 0.5210 (0.5210)	Prec@1 84.375 (84.375)
Epoch: [8/200][100/782]	LR: 0.01	Time 0.333 (0.374)	Data 0.005 (0.046)	Loss 0.5269 (0.4386)	Prec@1 81.250 (84.684)
Epoch: [8/200][200/782]	LR: 0.01	Time 0.337 (0.356)	Data 0.006 (0.027)	Loss 0.4914 (0.4483)	Prec@1 84.375 (84.072)
Epoch: [8/200][300/782]	LR: 0.01	Time 0.336 (0.349)	Data 0.005 (0.020)	Loss 0.2915 (0.4553)	Prec@1 90.625 (84.012)
Epoch: [8/200][400/782]	LR: 0.01	Time 0.336 (0.346)	Data 0.005 (0.017)	Loss 0.3272 (0.4551)	Prec@1 87.500 (84.083)
Epoch: [8/200][500/782]	LR: 0.01	Time 0.340 (0.345)	Data 0.005 (0.015)	Loss 0.3356 (0.4534)	Prec@1 84.375 (84.191)
Epoch: [8/200][600/782]	LR: 0.01	Time 0.340 (0.344)	Dat

KeyboardInterrupt: 